In [17]:
import awswrangler as wr
import boto3
from dotenv import load_dotenv
import pandas as pd
import os
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC




from concurrent.futures import ThreadPoolExecutor
from tempfile import mkdtemp

liftingcast_home = "https://liftingcast.com/"

number_of_worker_threads = 3

def get_webdirver(): 
    options = webdriver.ChromeOptions()
    options.add_argument("headless")
    options.add_argument("--log-level=3")
    options.add_experimental_option("excludeSwitches", ["enable-logging"])
    driver = webdriver.Chrome(
        options=options, service=ChromeService(ChromeDriverManager().install())
    )
    driver.implicitly_wait(2)
    return driver


load_dotenv()
region = os.environ.get("AWS_REGION")
lifter_table_name = os.environ.get("AWS_DYNAMO_LIFTER_TABLE_NAME")
lifter_update_table_name = os.environ.get("AWS_DYNAMO_LIFTER_UPDATE_TABLE_NAME")
boto3.setup_default_session(region_name=region)



In [18]:
d = get_webdirver()
d.get(liftingcast_home)
tables = d.find_elements(By.CLASS_NAME, "table")
upcoming_meets = None
recent_meets = None
for table in tables:
    head = table.find_element(By.XPATH, ("./thead"))
    if "Upcoming Meets" in head.text:
        upcoming_meets = table
    elif "Recent Meets" in head.text:
        recent_meets = table

meets = []
for meet_table in [upcoming_meets, recent_meets]:
    rows = meet_table.find_elements(By.XPATH, ("./tbody/tr"))
    for row in rows:
        [name, date] = row.find_elements(By.XPATH, ("./td"))
        meet_id = (
            name.find_element(By.XPATH, ("./a"))
            .get_attribute("href")
            .split("/")[4]
        )
        meets.append({"name": name.text, "date": date.text, "meet_id": meet_id})

d.quit()

In [19]:
def split_list_into_chunks(l, nchunks):
    for i in range(nchunks):
        yield l[i::nchunks]

meet_buckets =  split_list_into_chunks(meets, number_of_worker_threads)


In [ ]:
def fetch_lifters_from_meet(meet, driver):
    meet_url = f"https://liftingcast.com/meets/{meet['meet_id']}/roster"
    try:
        driver.get(meet_url)
        wait = WebDriverWait(driver, timeout=15)
        wait.until(
            EC.presence_of_element_located((By.CLASS_NAME, "meet-info-row"))
        )
    except Exception as e:
        print("EXCEPTION WHILE WAITING FOR " + meet["name"] + " TO LOAD")
        print(e)

    lifters = driver.find_elements(By.TAG_NAME, "a")
    result = []
    for lifter in lifters:
        lifter_id = lifter.get_attribute("href").split("/")[-1]
        result.append(
            {
                "lifter_name": lifter.text,
                "lifter_id": lifter_id,
                "meet_name": meet["name"],
                "meet_id": meet["meet_id"],
                "meet_date": meet["date"],
            }
        )
    print(f"fetched {len(result)} lifters for {meet['name']}")
    return result

def process_meets(meets):
    driver = get_webdirver()
    result_collector = []
    for meet in meets:
        result_collector.extend(fetch_lifters_from_meet(meet, driver))
    return result_collector

with ThreadPoolExecutor(number_of_worker_threads) as executor:
    futures = [
        executor.submit(process_meets, bucket) for bucket in meet_buckets
    ]
result = [f.result() for f in futures]
print(result)
lifters = result


In [25]:
lifters = [lifter for meet in result for lifter in meet]